In [2]:
from transformers import DistilBertTokenizer, DistilBertModel, DistilBertConfig
from transformers import BertTokenizer, BertModel, BertConfig

import torch
import numpy as np

import pickle
from tqdm import tqdm

from collections import Counter
import matplotlib.pyplot as plt

import logging
logging.getLogger("transformers.tokenization_utils").setLevel(logging.ERROR)

---
# Initialise Tokeniser & Model

In [5]:
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertModel.from_pretrained('distilbert-base-uncased')

model.eval()

I0224 12:16:56.094146 140035151705920 configuration_utils.py:254] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/distilbert-base-uncased-config.json from cache at /home/valentin/.cache/torch/transformers/a41e817d5c0743e29e86ff85edc8c257e61bc8d88e4271bb1b243b6e7614c633.587f67ec28c540f4294c9c2ac7dcf7841ff371aeb12cdeb6a17f69da39ad9452
I0224 12:16:56.096509 140035151705920 configuration_utils.py:290] Model config DistilBertConfig {
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 0,
  "dim": 768,
  "do_sample": false,
  "dropout": 0.1,
  "eos_token_ids": 0,
  "finetuning_task": null,
  "hidden_dim": 3072,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1"
  },
  "initializer_range": 0.02,
  "is_decoder": false,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1
  },
  "length_penalty": 1.0,
  "max_length": 20,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_he

DistilBertModel(
  (embeddings): Embeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (transformer): Transformer(
    (layer): ModuleList(
      (0): TransformerBlock(
        (dropout): Dropout(p=0.1, inplace=False)
        (attention): MultiHeadSelfAttention(
          (dropout): Dropout(p=0.1, inplace=False)
          (q_lin): Linear(in_features=768, out_features=768, bias=True)
          (k_lin): Linear(in_features=768, out_features=768, bias=True)
          (v_lin): Linear(in_features=768, out_features=768, bias=True)
          (out_lin): Linear(in_features=768, out_features=768, bias=True)
        )
        (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (ffn): FFN(
          (dropout): Dropout(p=0.1, inplace=False)
          (lin1): Linear(in_features=768, out_fea

## Defs convert email text to model input and then model output

In [6]:
def email_to_ids(email_body):
    return torch.tensor(tokenizer.encode(email_body, add_special_tokens=True))

def email_to_vec(email_body_ids, to_id_first=True):
    if to_id_first:
        input_ids = email_to_ids(email_body_ids)
    else:
        input_ids = email_body_ids
        
    input_ids = input_ids.unsqueeze(0)
    outputs = model(input_ids)
    return outputs
    
#     last_hidden_states = outputs[0][0]
#     return last_hidden_states.mean(0)  # mean vs sum

---
# Load emails and apply tokeniser and conversion to model input

In [7]:
with open("../../w3c-emails/emails.pkl", "rb") as handle:
    emails = pickle.load(handle)

In [8]:
texts = [e.body_raw for e in emails]

In [ ]:
tok_ids = [email_to_ids(t) for t in tqdm(texts)]
with open("emails_token_ids.pkl", "wb") as handle:
    pickle.dump(tok_ids, handle)
exit()

---
# Cut up emails longer than 512 tokens & group into batches
(start from here by loading converted emails from pkl)

In [ ]:
with open("emails_token_ids.pkl", "rb") as handle:
    ids = pickle.load(handle)

In [ ]:
cut_up = [id_tens.split(511) for id_tens in ids]

In [ ]:
with torch.no_grad():    
    vecs = [[email_to_vec(v, to_id_first=False) for v in tt] for tt in tqdm(cut_up[:5000])]
    
with open("vectors2.pkl", "wb") as handle:
    pickle.dump(vecs, handle)
    
exit()

In [ ]:
with open("vectors.pkl", "rb") as handle:
    vecs_loaded = pickle.load(handle)

In [ ]:
len(vecs_loaded), len(vecs_loaded[0]), vecs_loaded[0][0].shape

In [ ]:
lens = [len(t) for t in texts if len(t) < ]

plt.hist(lens, bins=30)

